# Knowledge Graphs - datasets quality review
### Analysis of the quality of top publishers at data.europa.eu
##### Patryk Rakus, Michał Tomczyk

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from utils import SPARQL_to_df
from FAIR import findability, accessibility, interoperability, reusability
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
connection = SPARQLWrapper("https://data.europa.eu/sparql")

In [3]:
PREFIXES = """
    PREFIX dcat: <http://www.w3.org/ns/dcat#>
    PREFIX odp:  <http://data.europa.eu/euodp/ontologies/ec-odp#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX void: <http://rdfs.org/ns/void#>
"""


Finding the top themes:

In [4]:
query = '''
SELECT ?theme (count(*) as ?DatasetsPublished) WHERE {
    ?datasetURI a dcat:Dataset;
    dcat:theme ?theme .
}
GROUP BY ?theme
ORDER BY DESC(?DatasetsPublished)
'''

In [5]:
df = SPARQL_to_df(connection, query, PREFIXES)

In [24]:
df.iloc[25:35]

,theme,DatasetsPublished
25,https://data.gov.ie/Environment,4089
26,http://eurovoc.europa.eu/6416,4068
27,http://inspire.ec.europa.eu/theme/nz,2674
28,http://standaarden.overheid.nl/owms/terms/Econ...,2672
29,http://standaarden.overheid.nl/owms/terms/Natu...,2660
30,https://data.gov.ie/Health,2377
31,http://standaarden.overheid.nl/owms/terms/Bestuur,2287
32,http://publications.europa.eu/resource/authori...,1731
33,http://standaarden.overheid.nl/owms/terms/Orga...,1426
34,http://inspire.ec.europa.eu/theme/sd,1328


In [25]:
df.iloc[25:35]['DatasetsPublished'].astype('Int64').sum()

np.int64(25312)

In [26]:
top_themes = list(df.iloc[25:35]['theme'])

In [27]:
newline = '\n'

In [28]:
query = f'''
VALUES ?theme {{
    {"".join(f"<{theme}>" + newline for theme in top_themes)}
    }}
    ?dataset a dcat:Dataset ;
           dcat:theme ?publisher .
'''

In [29]:
df_findability = findability.findability_aggregated_properties(connection, query, PREFIXES, 'theme')

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Virtuoso 42000 Error The estimated execution time 337 (sec) exceeds the limit of 300 (sec).\n\nSPARQL query:\n\n    PREFIX dcat: <http://www.w3.org/ns/dcat#>\n    PREFIX odp:  <http://data.europa.eu/euodp/ontologies/ec-odp#>\n    PREFIX dct: <http://purl.org/dc/terms/>\n    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n    PREFIX foaf: <http://xmlns.com/foaf/0.1/>\n    PREFIX void: <http://rdfs.org/ns/void#>\n\n    SELECT ?theme\n        (COUNT(DISTINCT ?dataset) AS ?totalDatasets)\n        (AVG(?keywordCount) AS ?avgKeywords)\n        ((100.0 * SUM(?hasTitle) / COUNT(DISTINCT ?dataset)) AS ?percentageWithTitle)\n        ((100.0 * SUM(?hasIssuedDate) / COUNT(DISTINCT ?dataset)) AS ?percentageWithIssuedDate)\n        ((100.0 * SUM(?hasLocation) / COUNT(DISTINCT ?dataset)) AS ?percentageWithLocation)\n        ((100.0 * SUM(?hasType) / COUNT(DISTINCT ?dataset)) AS ?percentageWith

In [11]:
df_findability

,publisher,totalDatasets,avgKeywords,percentageWithTitle,percentageWithIssuedDate,percentageWithLocation,percentageWithType,percentageWithPartOf
0,http://publications.europa.eu/resource/authori...,3662,1,100,100,89.131622064445658,0,100
1,https://www.bfs.admin.ch/,2904,1,100,100,98.588154269972452,0,0
2,https://org.belgif.be/id/CbeEstablishmentUnit/...,5660,1,100,97.54416961130742,100,0,0
3,https://opendata.schleswig-holstein.de/organiz...,15193,1,100,100,99.914434278944251,0.085565721055749,0
4,http://datos.gob.es/recurso/sector-publico/org...,8217,1,100,100,100,0,0
5,https://www.data.gv.at/katalog/organization/f0...,42544,1,100,100,0,0,0
6,https://data.gov.ie/organization/central-stati...,10699,1,100,100,0.028040003738667,0,0
7,https://www.data.gv.at/katalog/organization/89...,6151,1,100,100,0,0,0
8,http://standaarden.overheid.nl/owms/terms/Cent...,7740,1,100,0,0,0,0
9,http://publications.europa.eu/resource/authori...,9242,1,100,90.802856524561783,72.679073793551179,99.956719324821467,0


In [30]:
df_interoperability = interoperability.interoperability_aggregated_properties(connection, query, PREFIXES)

HTTPError: HTTP Error 504: Gateway Time-out

In [ ]:
df_interoperability

In [14]:
df_both_attributes = df_interoperability.merge(df_findability, on='publisher')

In [18]:
df_both_attributes

,publisher,totalDatasets_x,avgReferences,percentageWithLanguage,percentageWithAccessRights,percentageBeingVersionOf,percentageWithIdentifier,percentageWithRightsHolder,totalDatasets_y,avgKeywords,percentageWithTitle,percentageWithIssuedDate,percentageWithLocation,percentageWithType,percentageWithPartOf
0,https://org.belgif.be/id/CbeEstablishmentUnit/...,5660,1,100,100,0,100,99.681978798586572,5660,1,100,97.54416961130742,100,0,0
1,http://datos.gob.es/recurso/sector-publico/org...,8217,4.044541803577948,100,0,0,100,0,8217,1,100,100,100,0,0
2,https://www.bfs.admin.ch/,2904,1,100,0,0,100,0,2904,1,100,100,98.588154269972452,0,0
3,https://data.gov.ie/organization/central-stati...,10699,1,99.990653332087111,0,0,100,0,10699,1,100,100,0.028040003738667,0,0
4,http://publications.europa.eu/resource/authori...,3662,1.082195521572911,87.738940469688695,0,0,100,0,3662,1,100,100,89.131622064445658,0,100
5,https://opendata.schleswig-holstein.de/organiz...,15193,1,0.04607384979925,0.0394918712565,2.415586125189232,100,0,15193,1,100,100,99.914434278944251,0.085565721055749,0
6,https://www.data.gv.at/katalog/organization/f0...,42544,1,0,0,0,100,0,42544,1,100,100,0,0,0
7,https://www.data.gv.at/katalog/organization/89...,6151,1,0,0,0,100,0,6151,1,100,100,0,0,0
8,http://standaarden.overheid.nl/owms/terms/Cent...,7740,1,100,16.963824289405685,0,0,0,7740,1,100,0,0,0,0
9,http://publications.europa.eu/resource/authori...,9242,1,0.0324605063839,94.687297121835101,0,99.978359662410734,0,9242,1,100,90.802856524561783,72.679073793551179,99.956719324821467,0
